<div style="background-color: #04D7FD; padding: 20px; text-align: left;">
    <h1 style="color: #000000; font-size: 36px; margin: 0;">Data Processing for RAG with Data Prep Kit (Python)</h1>
    
</div>


## Before Running the notebook

Please complete [setting up python dev environment](./setup-python-dev-env.md)

## Overview

This notebook will process PDF documents as part of RAG pipeline

![](media/rag-overview-2.png)

This notebook will perform steps 1, 2, 3 and 4 in RAG pipeline.

Here are the processing steps:

- **pdf2parquet** : Extract text (in markdown format) from PDF and store them as parquet files
- **Exact Dedup**: Documents with exact content are filtered out
- **Chunk documents**: Split the PDFs into 'meaningful sections' (paragraphs, sentences ..etc)
- **Text encoder**: Convert chunks into vectors using embedding models

## Step-1: Configuration

In [2]:
from my_config import MY_CONFIG

In [3]:
## setup path to utils folder
import sys
sys.path.append('../utils')

## Step-2:  Data

We will use white papers  about LLMs.  

- [Granite Code Models](https://arxiv.org/abs/2405.04324)
- [Attention is all you need](https://arxiv.org/abs/1706.03762)

You can of course substite your own data below

### 2.1 - Download data

In [7]:
import os, sys
import shutil
from file_utils import download_file

shutil.rmtree(MY_CONFIG.INPUT_DATA_DIR, ignore_errors=True)
shutil.os.makedirs(MY_CONFIG.INPUT_DATA_DIR, exist_ok=True)
print ("✅ Cleared input directory")
 
download_file (url = 'https://arxiv.org/pdf/1706.03762', local_file = os.path.join(MY_CONFIG.INPUT_DATA_DIR, 'attention.pdf' ))
download_file (url = 'https://arxiv.org/pdf/2405.04324', local_file = os.path.join(MY_CONFIG.INPUT_DATA_DIR, 'granite.pdf' ))
download_file (url = 'https://arxiv.org/pdf/2405.04324', local_file = os.path.join(MY_CONFIG.INPUT_DATA_DIR, 'granite2.pdf' )) # duplicate


ModuleNotFoundError: No module named 'pandas'

### 2.2 - Set input/output path variables for the pipeline

In [4]:
import os, sys
import shutil

if not os.path.exists(MY_CONFIG.INPUT_DATA_DIR ):
    raise Exception (f"❌ Input folder MY_CONFIG.INPUT_DATA_DIR = '{MY_CONFIG.INPUT_DATA_DIR}' not found")

output_parquet_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '01_parquet_out')
output_exact_dedupe_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '02_dedupe_out')
output_chunk_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '03_chunk_out')
output_embeddings_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '04_embeddings_out')

## clear output folder
shutil.rmtree(MY_CONFIG.OUTPUT_FOLDER, ignore_errors=True)
shutil.os.makedirs(MY_CONFIG.OUTPUT_FOLDER, exist_ok=True)

print ("✅ Cleared output directory")

✅ Cleared output directory


## Step-3: pdf2parquet -  Convert data from PDF to Parquet

This step is reading the input folder containing all PDF files and ingest them in a parquet table using the [Docling package](https://github.com/DS4SD/docling).
The documents are converted into a JSON format which allows to easily chunk it in the later steps.



### 3.1 - Execute 

In [5]:
%%time 

from dpk_pdf2parquet.transform_python import Pdf2Parquet
from dpk_pdf2parquet.transform import pdf2parquet_contents_types

STAGE = 1 
print (f"🏃🏼 STAGE-{STAGE}: Processing input='{MY_CONFIG.INPUT_DATA_DIR}' --> output='{output_parquet_dir}'\n", flush=True)

result = Pdf2Parquet(input_folder= MY_CONFIG.INPUT_DATA_DIR,
                    output_folder= output_parquet_dir,
                    data_files_to_use=['.pdf'],
                    pdf2parquet_contents_type=pdf2parquet_contents_types.MARKDOWN,   # markdown
                    #    pdf2parquet_contents_type=pdf2parquet_contents_types.JSON   # JSON
                    ).transform()

if result == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception (f"❌ Stage:{STAGE}  failed")

🏃🏼 STAGE-1: Processing input='input' --> output='output/01_parquet_out'



11:11:18 INFO - pdf2parquet parameters are : {'batch_size': -1, 'artifacts_path': None, 'contents_type': <pdf2parquet_contents_types.MARKDOWN: 'text/markdown'>, 'do_table_structure': True, 'do_ocr': True, 'ocr_engine': <pdf2parquet_ocr_engine.EASYOCR: 'easyocr'>, 'bitmap_area_threshold': 0.05, 'pdf_backend': <pdf2parquet_pdf_backend.DLPARSE_V2: 'dlparse_v2'>, 'double_precision': 8}
11:11:18 INFO - pipeline id pipeline_id
11:11:18 INFO - code location None
11:11:18 INFO - data factory data_ is using local data access: input_folder - input output_folder - output/01_parquet_out
11:11:18 INFO - data factory data_ max_files -1, n_sample -1
11:11:18 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.pdf'], files to checkpoint ['.parquet']
11:11:18 INFO - orchestrator pdf2parquet started at 2025-03-14 11:11:18
11:11:18 INFO - Number of files is 3, source profile {'max_file_size': 2.112621307373047, 'min_file_size': 1.2146415710

✅ Stage:1 completed successfully
CPU times: user 1min 2s, sys: 1.84 s, total: 1min 4s
Wall time: 58.3 s


### 3.2 -  Inspect Generated output

Here we should see one entry per input file processed

In [6]:
from file_utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_parquet_dir)

# print ("Output dimensions (rows x columns)= ", output_df.shape)

output_df.head(5)

## To display certain columns
#parquet_df[['column1', 'column2', 'column3']].head(5)

Successfully read 3 parquet files with 3 total rows


,filename,contents,num_pages,num_tables,num_doc_elements,document_id,document_hash,ext,hash,size,date_acquired,pdf_convert_time,source_filename
0,attention.pdf,"Provided proper attribution is provided, Googl...",15,6,436,cebabb79-b5ba-4955-91bc-293c9f0052f1,2949302674760005271,pdf,5bbff6b2beac5f09f368a913932212e8d57953b3e3e86d...,45855,2025-03-14T11:11:35.158814,12.844160,attention.pdf
1,granite2.pdf,## Granite Code Models: A Family of Open Found...,28,21,324,540c13b9-50c6-4079-a214-4fb720d391ee,3127757990743433032,pdf,bf9642b9f719c6c7db1ad66a2992c96a755339ba9e793e...,136086,2025-03-14T11:12:13.016518,18.821193,granite2.pdf
2,granite.pdf,## Granite Code Models: A Family of Open Found...,28,21,324,78762d84-35f0-4bcb-8785-622a730c37ed,3127757990743433032,pdf,bf9642b9f719c6c7db1ad66a2992c96a755339ba9e793e...,136086,2025-03-14T11:11:54.146730,18.945178,granite.pdf


## Step-4: Eliminate Duplicate Documents

We have 2 duplicate documnets here : `granite.pdf` and `granite2.pdf`.

Note how the `hash` for these documents are same.

We are going to perform **de-dupe**

On the content of each document, a SHA256 hash is computed, followed by de-duplication of record having identical hashes.

[Dedupe transform documentation](https://github.com/data-prep-kit/data-prep-kit/blob/dev/transforms/universal/ededup/README.md)

### 4.1 - Execute 

In [7]:
%%time 

from dpk_ededup.transform_python import Ededup

STAGE = 2
print (f"🏃🏼 STAGE-{STAGE}: Processing input='{output_parquet_dir}' --> output='{output_exact_dedupe_dir}'\n", flush=True)

result = Ededup(input_folder=output_parquet_dir,
    output_folder=output_exact_dedupe_dir,
    ededup_doc_column="contents",
    ededup_doc_id_column="document_id"
    ).transform()

if result == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception (f"❌ Stage:{STAGE}  failed")

🏃🏼 STAGE-2: Processing input='output/01_parquet_out' --> output='output/02_dedupe_out'



11:12:13 INFO - exact dedup params are {'doc_column': 'contents', 'doc_id_column': 'document_id', 'use_snapshot': False, 'snapshot_directory': None}
11:12:13 INFO - pipeline id pipeline_id
11:12:13 INFO - code location None
11:12:13 INFO - data factory data_ is using local data access: input_folder - output/01_parquet_out output_folder - output/02_dedupe_out
11:12:13 INFO - data factory data_ max_files -1, n_sample -1
11:12:13 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
11:12:13 INFO - orchestrator ededup started at 2025-03-14 11:12:13
11:12:13 INFO - Number of files is 3, source profile {'max_file_size': 0.0434112548828125, 'min_file_size': 0.020546913146972656, 'total_file_size': 0.10735607147216797}
11:12:13 INFO - Starting from the beginning
11:12:13 INFO - Completed 1 files (33.33%) in 0.0 min
11:12:13 INFO - Completed 2 files (66.67%) in 0.0 min
11:12:13 INFO - Com

✅ Stage:2 completed successfully
CPU times: user 37.7 ms, sys: 3.92 ms, total: 41.6 ms
Wall time: 36.4 ms


### 4.2 - Inspect Generated output

We would see 2 documents: `attention.pdf`  and `granite.pdf`.  The duplicate `granite.pdf` has been filtered out!

In [8]:
from file_utils import read_parquet_files_as_df

input_df = read_parquet_files_as_df(output_parquet_dir)
output_df = read_parquet_files_as_df(output_exact_dedupe_dir)

# print ("Input data dimensions (rows x columns)= ", input_df.shape)
# print ("Output data dimensions (rows x columns)= ", output_df.shape)
print (f"Input files before exact dedupe : {input_df.shape[0]:,}")
print (f"Output files after exact dedupe : {output_df.shape[0]:,}")
print ("Duplicate files removed :  ", (input_df.shape[0] - output_df.shape[0]))

output_df.sample(min(3, output_df.shape[0]))

Successfully read 3 parquet files with 3 total rows
Successfully read 2 parquet files with 2 total rows
Input files before exact dedupe : 3
Output files after exact dedupe : 2
Duplicate files removed :   1


,filename,contents,num_pages,num_tables,num_doc_elements,document_id,document_hash,ext,hash,size,date_acquired,pdf_convert_time,source_filename,removed
0,attention.pdf,"Provided proper attribution is provided, Googl...",15,6,436,cebabb79-b5ba-4955-91bc-293c9f0052f1,2949302674760005271,pdf,5bbff6b2beac5f09f368a913932212e8d57953b3e3e86d...,45855,2025-03-14T11:11:35.158814,12.844160,attention.pdf,[]
1,granite.pdf,## Granite Code Models: A Family of Open Found...,28,21,324,78762d84-35f0-4bcb-8785-622a730c37ed,3127757990743433032,pdf,bf9642b9f719c6c7db1ad66a2992c96a755339ba9e793e...,136086,2025-03-14T11:11:54.146730,18.945178,granite.pdf,[]


##  Step-5: Doc chunks

Split the documents in chunks.

[Chunking transform documentation](https://github.com/data-prep-kit/data-prep-kit/blob/dev/transforms/language/doc_chunk/README.md)

**Experiment with chunking size to find the setting that works best for your documents**

### 5.1 - Execute 

In [9]:
%%time

from dpk_doc_chunk.transform_python import DocChunk

STAGE = 3
print (f"🏃🏼 STAGE-{STAGE}: Processing input='{output_exact_dedupe_dir}' --> output='{output_chunk_dir}'\n", flush=True)

result = DocChunk(input_folder=output_exact_dedupe_dir,
        output_folder=output_chunk_dir,
        doc_chunk_chunking_type= "li_markdown",
        # doc_chunk_chunking_type= "dl_json",
        doc_chunk_chunk_size_tokens = 128,  # default 128
        doc_chunk_chunk_overlap_tokens=30   # default 30
        ).transform()

if result == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception (f"❌ Stage:{STAGE}  failed")

🏃🏼 STAGE-3: Processing input='output/02_dedupe_out' --> output='output/03_chunk_out'



11:12:13 INFO - doc_chunk parameters are : {'chunking_type': 'li_markdown', 'content_column_name': 'contents', 'doc_id_column_name': 'document_id', 'output_chunk_column_name': 'contents', 'output_source_doc_id_column_name': 'source_document_id', 'output_jsonpath_column_name': 'doc_jsonpath', 'output_pageno_column_name': 'page_number', 'output_bbox_column_name': 'bbox', 'chunk_size_tokens': 128, 'chunk_overlap_tokens': 30, 'dl_min_chunk_len': None}
11:12:13 INFO - pipeline id pipeline_id
11:12:13 INFO - code location None
11:12:13 INFO - data factory data_ is using local data access: input_folder - output/02_dedupe_out output_folder - output/03_chunk_out
11:12:13 INFO - data factory data_ max_files -1, n_sample -1
11:12:13 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
11:12:13 INFO - orchestrator doc_chunk started at 2025-03-14 11:12:13
11:12:13 INFO - Number of files is 3,

✅ Stage:3 completed successfully
CPU times: user 656 ms, sys: 86.2 ms, total: 742 ms
Wall time: 736 ms


### 5.2 - Inspect Generated output

We would see documents are split into many chunks

In [10]:
from file_utils import read_parquet_files_as_df

input_df = read_parquet_files_as_df(output_exact_dedupe_dir)  ## for debug purposes
output_df = read_parquet_files_as_df(output_chunk_dir)

print (f"Files processed : {input_df.shape[0]:,}")
print (f"Chunks created : {output_df.shape[0]:,}")

# print ("Input data dimensions (rows x columns)= ", input_df.shape)
# print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(min(3, output_df.shape[0]))

Successfully read 2 parquet files with 2 total rows
Successfully read 2 parquet files with 61 total rows
Files processed : 2
Chunks created : 61


,filename,num_pages,num_tables,num_doc_elements,document_hash,ext,hash,size,date_acquired,pdf_convert_time,source_filename,removed,source_document_id,contents,document_id
14,attention.pdf,15,6,436,2949302674760005271,pdf,5bbff6b2beac5f09f368a913932212e8d57953b3e3e86d...,45855,2025-03-14T11:11:35.158814,12.844160,attention.pdf,[],cebabb79-b5ba-4955-91bc-293c9f0052f1,## 3.5 Positional Encoding\n\nSince our model ...,cb11d177cf91a1d4e6dfeefc101bb2b597aaf6254efdbc...
9,attention.pdf,15,6,436,2949302674760005271,pdf,5bbff6b2beac5f09f368a913932212e8d57953b3e3e86d...,45855,2025-03-14T11:11:35.158814,12.844160,attention.pdf,[],cebabb79-b5ba-4955-91bc-293c9f0052f1,## 3.2.1 Scaled Dot-Product Attention\n\nWe ca...,372f585509e5743de4e6603483cad7dfa3bd5bbdfa131c...
45,granite.pdf,28,21,324,3127757990743433032,pdf,bf9642b9f719c6c7db1ad66a2992c96a755339ba9e793e...,136086,2025-03-14T11:11:54.146730,18.945178,granite.pdf,[],78762d84-35f0-4bcb-8785-622a730c37ed,## 6.1.1 HumanEvalSynthesize: Multilingual Cod...,88dab0e6c051bffc8c4bcf9c070da2d1fe75c0a214b41b...


## Step-6:   Calculate Embeddings for Chunks

we will calculate embeddings for each chunk using an open source embedding model

[Embeddings / Text Encoder documentation](https://github.com/data-prep-kit/data-prep-kit/blob/dev/transforms/language/text_encoder/README.md)

### 6.1 - Execute

In [11]:
%%time 

from dpk_text_encoder.transform_python import TextEncoder

STAGE  = 4
print (f"🏃🏼 STAGE-{STAGE}: Processing input='{output_chunk_dir}' --> output='{output_embeddings_dir}'\n", flush=True)


result = TextEncoder(input_folder= output_chunk_dir, 
               output_folder= output_embeddings_dir, 
               text_encoder_model_name = MY_CONFIG.EMBEDDING_MODEL
               ).transform()
if result == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception (f"❌ Stage:{STAGE}  failed")

🏃🏼 STAGE-4: Processing input='output/03_chunk_out' --> output='output/04_embeddings_out'



11:12:14 INFO - text_encoder parameters are : {'content_column_name': 'contents', 'output_embeddings_column_name': 'embeddings', 'model_name': 'ibm-granite/granite-embedding-30m-english'}
11:12:14 INFO - pipeline id pipeline_id
11:12:14 INFO - code location None
11:12:14 INFO - data factory data_ is using local data access: input_folder - output/03_chunk_out output_folder - output/04_embeddings_out
11:12:14 INFO - data factory data_ max_files -1, n_sample -1
11:12:14 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
11:12:14 INFO - orchestrator text_encoder started at 2025-03-14 11:12:14
11:12:14 INFO - Number of files is 2, source profile {'max_file_size': 0.04574298858642578, 'min_file_size': 0.028684616088867188, 'total_file_size': 0.07442760467529297}
11:12:16 INFO - Completed 1 files (50.0%) in 0.003 min
11:12:17 INFO - Completed 2 files (100.0%) in 0.006 min
11:12:17 INF

✅ Stage:4 completed successfully
CPU times: user 1.03 s, sys: 164 ms, total: 1.19 s
Wall time: 3.22 s


### 6.2 - Inspect Generated output

In [12]:
from file_utils import read_parquet_files_as_df

input_df = read_parquet_files_as_df(output_chunk_dir)
output_df = read_parquet_files_as_df(output_embeddings_dir)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(min(3, output_df.shape[0]))

Successfully read 2 parquet files with 61 total rows
Successfully read 2 parquet files with 61 total rows
Input data dimensions (rows x columns)=  (61, 15)
Output data dimensions (rows x columns)=  (61, 16)


,filename,num_pages,num_tables,num_doc_elements,document_hash,ext,hash,size,date_acquired,pdf_convert_time,source_filename,removed,source_document_id,contents,document_id,embeddings
0,attention.pdf,15,6,436,2949302674760005271,pdf,5bbff6b2beac5f09f368a913932212e8d57953b3e3e86d...,45855,2025-03-14T11:11:35.158814,12.84416,attention.pdf,[],cebabb79-b5ba-4955-91bc-293c9f0052f1,"Provided proper attribution is provided, Googl...",cb3a9de8b5457dde64115f2fcd6e5b4093b228499af4ac...,"[-0.054784447, 0.049434207, 0.0080653345, 0.00..."
7,attention.pdf,15,6,436,2949302674760005271,pdf,5bbff6b2beac5f09f368a913932212e8d57953b3e3e86d...,45855,2025-03-14T11:11:35.158814,12.84416,attention.pdf,[],cebabb79-b5ba-4955-91bc-293c9f0052f1,## 3.2 Attention\n\nAn attention function can ...,bac7e7f7e0121639aa67e546cc42f16c1b1b3a4c5083f8...,"[0.021459015, 0.05516718, -0.041755836, 0.0438..."
5,attention.pdf,15,6,436,2949302674760005271,pdf,5bbff6b2beac5f09f368a913932212e8d57953b3e3e86d...,45855,2025-03-14T11:11:35.158814,12.84416,attention.pdf,[],cebabb79-b5ba-4955-91bc-293c9f0052f1,## 3 Model Architecture\n\nMost competitive ne...,b8ed30a0030817a11d373ce52e26bf6edb0a713571efc9...,"[-0.018718036, -0.01040597, 0.047386818, 0.010..."


## Step-7: Copy output to final output dir

In [13]:
import shutil

shutil.rmtree(MY_CONFIG.OUTPUT_FOLDER_FINAL, ignore_errors=True)
shutil.copytree(src=output_embeddings_dir, dst=MY_CONFIG.OUTPUT_FOLDER_FINAL)

print (f"✅ Copied output from '{output_embeddings_dir}' --> '{MY_CONFIG.OUTPUT_FOLDER_FINAL}'")

✅ Copied output from 'output/04_embeddings_out' --> 'output/output_final'
